# Projet Deep

In [1]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms

print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
caltech101_path = 'caltech-101/101_ObjectCategories'

La version de torch est :  2.1.1+cu121
Le calcul GPU est disponible ?  False


## Création des jeux d'entrainement et de test

In [2]:



def getDataLoader(data_path,size,mean,std,train_ratio,batch_size):

    ###################################    Train loader    #################################################""

    # Transformation des images
    transform = transforms.Compose([
        transforms.RandomResizedCrop(size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),  # Rotation aléatoire dans la plage de -10 à 10 degrés
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    # Charger le jeu de données avec ImageFolder
    caltech101_train_dataset = datasets.ImageFolder(root=data_path, transform=transform)
    # Utiliser DataLoader pour créer un itérateur sur le jeu de données
    #caltech101_train_loader = DataLoader(caltech101_train_dataset, shuffle=True)
    #print(len(caltech101_train_loader))

    train_length = int(len(caltech101_train_dataset) * train_ratio)
    indices = list(range(len(caltech101_train_dataset)))
    train_indices = indices[:train_length]
    test_indices = indices[train_length:]

    ds_train = Subset(caltech101_train_dataset, train_indices)
    train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)

    ###################################    Test loader    #################################################""

    transform = transforms.Compose([
        transforms.RandomResizedCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    # Charger le jeu de données avec ImageFolder
    caltech101_test_dataset = datasets.ImageFolder(root=data_path, transform=transform)

    ds_test = Subset(caltech101_test_dataset, test_indices)
    test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

    ################    Info  #############################
    classes = caltech101_train_dataset.classes
    total_images = len(caltech101_train_dataset)
    print("Classes:", len(classes))
    print("Nombre total d'images:", total_images)  
    print("Train:", len(train_loader)," Test:", len(test_loader))  

    return train_loader, test_loader


## Fonction d'entrainement et de test

In [3]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_iterator = iter(train_loader)  # Create an iterator for the training loader

    for images, labels in train_iterator:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


# Fonction de test
def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        test_iterator = iter(test_loader)  
        for images, labels in test_iterator:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    average_loss = running_loss / len(test_loader)
    return accuracy, average_loss


In [4]:
def launchModel(model,train_loader,test_loader,num_epochs,criterion,optimizer):
    for epoch in range(num_epochs):
        train(model, train_loader, criterion, optimizer)
        if epoch % 3 == 0: 
            accuracy, loss = test(model, test_loader, criterion)
            print(f'Epoch [{epoch+1}/{num_epochs}] - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}')

    # Évaluation finale sur les données de test
    final_accuracy, final_loss = test(model, test_loader, criterion)
    print(f'Final Accuracy: {final_accuracy:.4f}, Final Loss: {final_loss:.4f}')

# Models

## Resnet 18

In [5]:

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
TRAIN_RATIO = 0.9
batch_size = 64
size = (224,224)
train_loader,test_loader = getDataLoader(caltech101_path,size,mean,std,TRAIN_RATIO,batch_size)
#weights = models.ResNet18_Weights.DEFAULT

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Fixer les paramètres pré-entraînés (ils ne seront pas optimisés)
for param in model.parameters():
    param.requires_grad = False

# Modifier le dernier bloc entièrement connecté pour le nombre de classes correct
num_classes = 101
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Définir un optimiseur ne modifiant que les paramètres nouvellement ajoutés
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

launchModel(model,train_loader,test_loader,num_epochs,criterion,optimizer)



Classes: 101
Nombre total d'images: 8677
Train: 123  Test: 14


KeyboardInterrupt: 

## Alexnet

In [7]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
TRAIN_RATIO = 0.9
batch_size = 64
size = (224, 224)
train_loader, test_loader = getDataLoader(caltech101_path, size, mean, std, TRAIN_RATIO, batch_size)
model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)

# Fixer les paramètres pré-entraînés (ils ne seront pas optimisés)
for param in model.parameters():
    param.requires_grad = False

# Modifier la dernière couche entièrement connectée pour le nombre de classes correct
num_classes = 101
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Définir un optimiseur ne modifiant que les paramètres nouvellement ajoutés
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

launchModel(model, train_loader, test_loader, num_epochs, criterion, optimizer)




Classes: 101
Nombre total d'images: 8677
Train: 245  Test: 28
Epoch [1/100] - Accuracy: 0.0138, Loss: 12.4204
Epoch [4/100] - Accuracy: 0.0127, Loss: 15.8411


KeyboardInterrupt: 

## squeezenet1_0

In [10]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
TRAIN_RATIO = 0.9
batch_size = 64
size = (224, 224)
train_loader, test_loader = getDataLoader(caltech101_path, size, mean, std, TRAIN_RATIO, batch_size)
model = models.squeezenet1_0(weights=models.SqueezeNet1_0_Weights.DEFAULT)
# Fixer les paramètres pré-entraînés (ils ne seront pas optimisés)
for param in model.parameters():
    param.requires_grad = False

# Modifier la dernière couche entièrement connectée pour le nombre de classes correct
num_classes = 101
model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
model.num_classes = num_classes

# Définir un optimiseur ne modifiant que les paramètres nouvellement ajoutés
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

launchModel(model, train_loader, test_loader, num_epochs, criterion, optimizer)

Classes: 101
Nombre total d'images: 8677
Train: 123  Test: 14
SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expa

KeyboardInterrupt: 

## vgg16

In [12]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
TRAIN_RATIO = 0.9
batch_size = 64
size = (224, 224)
train_loader, test_loader = getDataLoader(caltech101_path, size, mean, std, TRAIN_RATIO, batch_size)
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
# Fixer les paramètres pré-entraînés (ils ne seront pas optimisés)
for param in model.parameters():
    param.requires_grad = False

# Modifier la dernière couche entièrement connectée pour le nombre de classes correct
num_classes = 101
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Définir un optimiseur ne modifiant que les paramètres nouvellement ajoutés
optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

launchModel(model, train_loader, test_loader, num_epochs, criterion, optimizer)

Classes: 101
Nombre total d'images: 8677
Train: 123  Test: 14


KeyboardInterrupt: 

## densenet161

In [13]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
TRAIN_RATIO = 0.9
batch_size = 64
size = (224, 224)
train_loader, test_loader = getDataLoader(caltech101_path, size, mean, std, TRAIN_RATIO, batch_size)
model = models.densenet161(weights=models.DenseNet161_Weights.DEFAULT)
# Fixer les paramètres pré-entraînés (ils ne seront pas optimisés)
for param in model.parameters():
    param.requires_grad = False

# Modifier la dernière couche entièrement connectée pour le nombre de classes correct
num_classes = 101
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Définir un optimiseur ne modifiant que les paramètres nouvellement ajoutés
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

launchModel(model, train_loader, test_loader, num_epochs, criterion, optimizer)

Classes: 101
Nombre total d'images: 8677
Train: 123  Test: 14


Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /home/etud/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:02<00:00, 47.2MB/s] 


KeyboardInterrupt: 

## inception_v3

In [14]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
TRAIN_RATIO = 0.9
batch_size = 64
size = (299, 299)
train_loader, test_loader = getDataLoader(caltech101_path, size, mean, std, TRAIN_RATIO, batch_size)
model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
# Fixer les paramètres pré-entraînés (ils ne seront pas optimisés)
for param in model.parameters():
    param.requires_grad = False

# Modifier la dernière couche entièrement connectée pour le nombre de classes correct
num_classes = 101
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Définir un optimiseur ne modifiant que les paramètres nouvellement ajoutés
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

launchModel(model, train_loader, test_loader, num_epochs, criterion, optimizer)

Classes: 101
Nombre total d'images: 8677
Train: 123  Test: 14


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/etud/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:02<00:00, 42.2MB/s] 


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not InceptionOutputs

## Annexe

In [ ]:
import matplotlib.pyplot as plt

imgs, labs = next(iter(caltech101_train_loader))
HEIGHT, WIDTH = imgs.shape[2], imgs.shape[3]  # Taille de l'image

# Dénormalisation de l'image
def unnormalize(img):
    if img.dim() == 2 or ((img.dim() == 3) and (img.size()[0] == 1)):
        return img * std[0] + mean[0]
    return img * img.new(std).view(3, 1, 1) + img.new(mean).view(3, 1, 1)

# Visualisation de la première image
print(imgs.size())
img = unnormalize(imgs[0])  # Pour retrouver l'image d'origine (avant normalisation)
fig = plt.figure(figsize=(8, 8))
plt.imshow(img.permute(1, 2, 0).squeeze(), cmap='Greys_r')  # Permutter les dimensions pour utiliser imshow correctement
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold

def train(model, train_loader, criterion, optimizer):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    average_loss = running_loss / len(test_loader)
    return accuracy, average_loss

def cross_validation(data_path, mean, std, num_classes, num_epochs, k_folds=5, batch_size=128, learning_rate=0.001):
    transform = transforms.Compose([
        transforms.RandomResizedCrop((300, 200)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    caltech101_dataset = datasets.ImageFolder(root=data_path, transform=transform)
    indices = list(range(len(caltech101_dataset)))

    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_indices, test_indices) in enumerate(kfold.split(indices)):
        print(f'Fold {fold + 1}/{k_folds}')

        ds_train = Subset(caltech101_dataset, train_indices)
        ds_test = Subset(caltech101_dataset, test_indices)

        train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

        model = models.resnet18(pretrained=True)
        model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            train(model, train_loader, criterion, optimizer)
            accuracy, loss = test(model, test_loader, criterion)
            print(f'Epoch [{epoch + 1}/{num_epochs}] - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}')

        final_accuracy, final_loss = test(model, test_loader, criterion)
        print(f'Fold {fold + 1} - Final Accuracy: {final_accuracy:.4f}, Final Loss: {final_loss:.4f}')

if __name__ == "__main__":
    data_path = 'caltech-101/101_ObjectCategories'
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    num_classes = 101
    num_epochs = 5
    k_folds = 5
    batch_size = 128
    learning_rate = 0.001

    cross_validation(data_path, mean, std, num_classes, num_epochs, k_folds, batch_size, learning_rate)
